## Implementing the aggregated features from the paper "Feature engineering strategies for creditcard fraud detection"

> Estimate the probability of a transaction being fraud based on analysing customer patterns and recent fraudelent behaviour

<img src="transaction_flow.PNG" width="800" height="600" alt="Alternative text" />

#### What is the goal of credit card fraud detection?

In [ ]:
import pandas as pd

In [ ]:
data = {"TrxId":[1,2,3,4,5,6,7],
        "CardId":[1,1,1,1,1,1,1],
        "Time":["01/01/15 18:20","01/01/15 20:35","01/01/15 22:30",
                "02/01/15 00:50","02/01/15 19:18","02/01/15 23:45",
                "03/01/15 00:00"],
        "Type":["POS","POS","ATM","POS","POS","POS","POS"],
        "Country":["Luxembourg","Luxembourg","Luxembourg",
                   "Germany","Germany","Germany","Luxembourg"],
        "Amount":[250,400,250,50,100,150,10]
       }

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

,TrxId,CardId,Time,Type,Country,Amount
0,1,1,01/01/15 18:20,POS,Luxembourg,250
1,2,1,01/01/15 20:35,POS,Luxembourg,400
2,3,1,01/01/15 22:30,ATM,Luxembourg,250
3,4,1,02/01/15 00:50,POS,Germany,50
4,5,1,02/01/15 19:18,POS,Germany,100
5,6,1,02/01/15 23:45,POS,Germany,150
6,7,1,03/01/15 00:00,POS,Luxembourg,10


In [ ]:
df['Time'] = pd.to_datetime(df['Time'],  infer_datetime_format=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   TrxId    7 non-null      int64         
 1   CardId   7 non-null      int64         
 2   Time     7 non-null      datetime64[ns]
 3   Type     7 non-null      object        
 4   Country  7 non-null      object        
 5   Amount   7 non-null      int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 464.0+ bytes


In [ ]:
df

,TrxId,CardId,Time,Type,Country,Amount
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250
3,4,1,2015-02-01 00:50:00,POS,Germany,50
4,5,1,2015-02-01 19:18:00,POS,Germany,100
5,6,1,2015-02-01 23:45:00,POS,Germany,150
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10


In [ ]:
print("Day:",df['Time'][3].day)
print("Month",df['Time'][3].month)
print("Year",df['Time'][3].year)
print("hour:",df['Time'][3].hour)
print("Minutes",df['Time'][3].minute)

Day: 1
Month 2
Year 2015
hour: 0
Minutes 50


In [ ]:
df_agg1 = df.set_index("Time").groupby("CardId").rolling("24h", min_periods=0, closed="left").agg({"Amount":"count"}).reset_index()

In [ ]:
df_agg1 = df_agg1.drop(columns='CardId')
df_agg1 = df_agg1.rename(columns={"Amount":"Xa1"})
df_agg1

,Time,Xa1
0,2015-01-01 18:20:00,0.0
1,2015-01-01 20:35:00,1.0
2,2015-01-01 22:30:00,2.0
3,2015-02-01 00:50:00,0.0
4,2015-02-01 19:18:00,1.0
5,2015-02-01 23:45:00,2.0
6,2015-03-01 00:00:00,0.0


In [ ]:
df_final = df.merge(df_agg1, on = "Time")

In [ ]:
df_final

,TrxId,CardId,Time,Type,Country,Amount,Xa1
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250,0.0
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400,1.0
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250,2.0
3,4,1,2015-02-01 00:50:00,POS,Germany,50,0.0
4,5,1,2015-02-01 19:18:00,POS,Germany,100,1.0
5,6,1,2015-02-01 23:45:00,POS,Germany,150,2.0
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10,0.0


In [ ]:
df_agg2 = df.set_index("Time").groupby("CardId").rolling("24h", min_periods=0, closed="left").agg({"Amount":"sum"}).reset_index()
df_agg2 = df_agg2.drop(columns='CardId')
df_agg2 = df_agg2.rename(columns={"Amount":"Xa2"})
df_agg2

,Time,Xa2
0,2015-01-01 18:20:00,0.0
1,2015-01-01 20:35:00,250.0
2,2015-01-01 22:30:00,650.0
3,2015-02-01 00:50:00,0.0
4,2015-02-01 19:18:00,50.0
5,2015-02-01 23:45:00,150.0
6,2015-03-01 00:00:00,0.0


In [ ]:
def get_aggregated_features(df, agg_period, time_column_name, groupby_on, agg_on, is_time_format_req=False):
    """
    This function provides aggregated features for credit card transaction fraud detection
    as explained in the paper "Feature engineering strategies for creditcard fraud detection"

    Args:
        df (pandas.DataFrame): Transactional data
        agg_period (str): time in hours used for aggregration e.g., "24h"
        time_column_name (str): name of the column represeting the time feature
        groupby_on (str): column name on which to perform the group by e.g., "CardId", "AccountNumber"
        agg_on (str): column name on which to perform aggregration e.g., "Amount"

    Returns:
        pandas.DataFrame: Transactional data with aggregated features appended as columns

    Usage:
    >>> df_agg = get_aggregated_features(df, agg_period="24h", time_column_name="Time", groupby_on="CardId", agg_on="Amount")
    """
    if is_time_format_req:
        # Change here according to the input format, if time column do not have "datetime" datatype
        df[time_column_name] = pd.to_datetime(df[time_column_name], format="mixed", dayfirst=True)
    #--------------------get the aggregated feature-1---------------------
    df_agg1 = df.set_index(time_column_name) \
              .groupby(groupby_on) \
              .rolling(agg_period, min_periods=0, closed="left") \
              .agg({agg_on:"count"}).reset_index()

    df_agg1 = df_agg1.drop(columns=groupby_on)
    df_agg1 = df_agg1.rename(columns={agg_on:"Xa1"})
    df_f1 = df.merge(df_agg1, on = time_column_name)
    #-------------------get the aggregated feature-2----------------------
    df_agg2 = df.set_index(time_column_name) \
              .groupby(groupby_on) \
              .rolling(agg_period, min_periods=0, closed="left") \
              .agg({agg_on:"sum"}).reset_index()

    df_agg2 = df_agg2.drop(columns=groupby_on)
    df_agg2 = df_agg2.rename(columns={agg_on:"Xa2"})
    df_f2 = df_f1.merge(df_agg2, on = time_column_name)
    #------------------ return the final result dataframe-----------------
    return df_f2

In [ ]:
df

,TrxId,CardId,Time,Type,Country,Amount
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250
3,4,1,2015-02-01 00:50:00,POS,Germany,50
4,5,1,2015-02-01 19:18:00,POS,Germany,100
5,6,1,2015-02-01 23:45:00,POS,Germany,150
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10


In [ ]:
# Test the function above
df_agg = get_aggregated_features(df, agg_period="24h", time_column_name="Time", groupby_on="CardId", agg_on="Amount")
df_agg

,TrxId,CardId,Time,Type,Country,Amount,Xa1,Xa2
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250,0.0,0.0
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400,1.0,250.0
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250,2.0,650.0
3,4,1,2015-02-01 00:50:00,POS,Germany,50,0.0,0.0
4,5,1,2015-02-01 19:18:00,POS,Germany,100,1.0,50.0
5,6,1,2015-02-01 23:45:00,POS,Germany,150,2.0,150.0
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10,0.0,0.0


In [ ]:
# Ref: https://www.codingem.com/python-how-to-get-all-combinations-of-a-list/
from itertools import chain, combinations
def powerset(items):
    # Ref: https://math.stackexchange.com/questions/161565/what-is-the-total-number-of-combinations-of-5-items-together-when-there-are-no-d
    l_items = list(items)
    return chain.from_iterable(combinations(l_items, r) for r in range(1,len(l_items) + 1))
def get_groupby_combinations(unique_id, features):
    """
    This function returns the combinations for the groupby to create additional interaction features

    Args:
        unique_id (str): unique_id usually the creditcard id or account number used to generate the Xa1 and Xa2 features
        features (list): List of features

    Returns:
        list: List of combinations to be used for creating the addditional features

    """
    out_list = []
    powerset_combination = powerset(features)
    for combination in powerset_combination:
        tmp_list = [unique_id]
        tmp_list.extend(combination)
        out_list.append(tmp_list)
    return out_list
unique_id = "TrxId"
features = ["Type", "Country"]
print(get_groupby_combinations(unique_id, features))

[['TrxId', 'Type'], ['TrxId', 'Country'], ['TrxId', 'Type', 'Country']]


In [ ]:
def get_aggregated_features_v1(df, agg_period, time_column_name, groupby_on, agg_on, features, is_time_format_req=False):
    """
    This function provides aggregated features for credit card transaction fraud detection
    as explained in the paper "Feature engineering strategies for creditcard fraud detection"

    Args:
        df (pandas.DataFrame): Transactional data
        agg_period (str): time in hours used for aggregration e.g., "24h"
        time_column_name (str): name of the column represeting the time feature
        groupby_on (str): column name on which to perform the group by e.g., "CardId", "AccountNumber"
        agg_on (str): column name on which to perform aggregration e.g., "Amount"
        features (list): List of useful transactional features to generate additional combinational features
        is_time_format_req (bool): If the specific time format conversion is required for time feature in the dataframe

    Returns:
        pandas.DataFrame: Transactional data with aggregated features appended as columns

    Usage:
    >>> df_agg = get_aggregated_features(df, agg_period="24h", time_column_name="Time", groupby_on="CardId", agg_on="Amount",
                                         features=["Type", "Country"])
    """
    if is_time_format_req:
        # Change here according to the input format, if time column do not have "datetime" datatype
        df[time_column_name] = pd.to_datetime(df[time_column_name], format="mixed", dayfirst=True)
    #--------------------get the aggregated feature-1---------------------
    df_agg1 = df.set_index(time_column_name) \
              .groupby(groupby_on) \
              .rolling(agg_period, min_periods=0, closed="left") \
              .agg({agg_on:"count"}).reset_index()

    df_agg1 = df_agg1.drop(columns=groupby_on)
    df_agg1 = df_agg1.rename(columns={agg_on:"Xa1"})
    df_f1 = df.merge(df_agg1, on = time_column_name)
    #-------------------get the aggregated feature-2----------------------
    df_agg2 = df.set_index(time_column_name) \
              .groupby(groupby_on) \
              .rolling(agg_period, min_periods=0, closed="left") \
              .agg({agg_on:"sum"}).reset_index()

    df_agg2 = df_agg2.drop(columns=groupby_on)
    df_agg2 = df_agg2.rename(columns={agg_on:"Xa2"})
    df_f2 = df_f1.merge(df_agg2, on = time_column_name)
    df_agg_final = df_f2
    #---------------------get the interaction features--------------------
    # get the list of possible combinations for group by
    feat_comb_list = get_groupby_combinations(unique_id=groupby_on, features=features)
    k = 0
    for combination in feat_comb_list:
        # Get the count feature
        df_agg_tmp1 = df.set_index(time_column_name) \
              .groupby(combination) \
              .rolling(agg_period, min_periods=0, closed="left") \
              .agg({agg_on:"count"}).reset_index()

        df_agg_tmp1 = df_agg_tmp1.drop(columns=combination)
        df_agg_tmp1 = df_agg_tmp1.rename(columns={agg_on:f"Xa{k+3}"})
        df_agg_final = df_agg_final.merge(df_agg_tmp1, on = time_column_name)
        # get the sum feature
        df_agg_tmp2 = df.set_index(time_column_name) \
              .groupby(combination) \
              .rolling(agg_period, min_periods=0, closed="left") \
              .agg({agg_on:"sum"}).reset_index()

        df_agg_tmp2 = df_agg_tmp2.drop(columns=combination)
        df_agg_tmp2 = df_agg_tmp2.rename(columns={agg_on:f"Xa{k+4}"})
        df_agg_final = df_agg_final.merge(df_agg_tmp2, on = time_column_name)
        # Increment the value of k by 2
        k = k+2
    #------------------ return the final result dataframe-----------------
    return df_agg_final

In [ ]:
get_aggregated_features_v1(df, agg_period="24h", time_column_name="Time", groupby_on="CardId", agg_on="Amount",
                           features=["Type", "Country"])

,TrxId,CardId,Time,Type,Country,Amount,Xa1,Xa2,Xa3,Xa4,Xa5,Xa6,Xa7,Xa8
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400,1.0,250.0,1.0,250.0,1.0,250.0,1.0,250.0
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250,2.0,650.0,0.0,0.0,2.0,650.0,0.0,0.0
3,4,1,2015-02-01 00:50:00,POS,Germany,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,2015-02-01 19:18:00,POS,Germany,100,1.0,50.0,1.0,50.0,1.0,50.0,1.0,50.0
5,6,1,2015-02-01 23:45:00,POS,Germany,150,2.0,150.0,2.0,150.0,2.0,150.0,2.0,150.0
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Description for the generated features:**
- `Xa1`: It shows the `total number of transactions` done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa2`: It shows the `total sum amount` for the total number of transactions done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa3`: It shows the `total number of transactions` for the given `type of transaction` done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa4`: It shows the `total sum amount` for the total number of transactions for the given `type of transaction` done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa5`: It shows the `total number of transactions` for the given `country` done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa6`: It shows the `total sum amount` for the total number of transactions for the given `country` done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa7`: It shows the `total number of transactions` for the given [`POS`,`country`] done in the last `24 hours` by the card holder with `CardId` as 1
- `Xa8`: It shows the `total sum amount` for the total number of transactions for the given [`POS`,`country`] done in the last `24 hours` by the card holder with `CardId` as 1

In [ ]:
from datetime import datetime
s1 = '2015-01-01 00:50:00' # 2015-01-02 00:50:00
s2 = '2015-01-02 00:49:00' # 2015-01-02 23:45:00
format = '%Y-%m-%d %H:%M:%S'
time = datetime.strptime(s2, format) - datetime.strptime(s1, format)
duration_in_s = time.total_seconds()
days  = divmod(duration_in_s, 86400)
hours = divmod(duration_in_s, 3600)
minutes = minutes = divmod(hours[1], 60)
seconds = seconds = divmod(minutes[1], 1)
print(f"{days[0]} days, {hours[0]} hours, {minutes[0]} minutes and {seconds[0]} seconds")

0.0 days, 23.0 hours, 59.0 minutes and 0.0 seconds


In [ ]:
df['Time'][5]

Timestamp('2015-02-01 23:45:00')

In [ ]:
1380+60

1440

In [ ]:
df['Time'][5] - pd.Timedelta(1440, 'minutes')

Timestamp('2015-01-31 23:45:00')

## For `agg_period` = 12h

In [ ]:
get_aggregated_features_v1(df, agg_period="12h", time_column_name="Time", groupby_on="CardId", agg_on="Amount",
                           features=["Type", "Country"])

,TrxId,CardId,Time,Type,Country,Amount,Xa1,Xa2,Xa3,Xa4,Xa5,Xa6,Xa7,Xa8
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400,1.0,250.0,1.0,250.0,1.0,250.0,1.0,250.0
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250,2.0,650.0,0.0,0.0,2.0,650.0,0.0,0.0
3,4,1,2015-02-01 00:50:00,POS,Germany,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,2015-02-01 19:18:00,POS,Germany,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1,2015-02-01 23:45:00,POS,Germany,150,1.0,100.0,1.0,100.0,1.0,100.0,1.0,100.0
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## For `agg_period` = 6h

In [ ]:
get_aggregated_features_v1(df, agg_period="6h", time_column_name="Time", groupby_on="CardId", agg_on="Amount",
                           features=["Type", "Country"])

,TrxId,CardId,Time,Type,Country,Amount,Xa1,Xa2,Xa3,Xa4,Xa5,Xa6,Xa7,Xa8
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400,1.0,250.0,1.0,250.0,1.0,250.0,1.0,250.0
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250,2.0,650.0,0.0,0.0,2.0,650.0,0.0,0.0
3,4,1,2015-02-01 00:50:00,POS,Germany,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,2015-02-01 19:18:00,POS,Germany,100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,1,2015-02-01 23:45:00,POS,Germany,150,1.0,100.0,1.0,100.0,1.0,100.0,1.0,100.0
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
get_aggregated_features_v1(df, agg_period="48h", time_column_name="Time", groupby_on="CardId", agg_on="Amount",
                           features=["Type", "Country"])

,TrxId,CardId,Time,Type,Country,Amount,Xa1,Xa2,Xa3,Xa4,Xa5,Xa6,Xa7,Xa8
0,1,1,2015-01-01 18:20:00,POS,Luxembourg,250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,1,2015-01-01 20:35:00,POS,Luxembourg,400,1.0,250.0,1.0,250.0,1.0,250.0,1.0,250.0
2,3,1,2015-01-01 22:30:00,ATM,Luxembourg,250,2.0,650.0,0.0,0.0,2.0,650.0,0.0,0.0
3,4,1,2015-02-01 00:50:00,POS,Germany,50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,1,2015-02-01 19:18:00,POS,Germany,100,1.0,50.0,1.0,50.0,1.0,50.0,1.0,50.0
5,6,1,2015-02-01 23:45:00,POS,Germany,150,2.0,150.0,2.0,150.0,2.0,150.0,2.0,150.0
6,7,1,2015-03-01 00:00:00,POS,Luxembourg,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
